# 1. Instalar MLflow

In [ ]:
# !pip install mlflow

# 2. Lanzar el servidor de MLflow

Para comenzar, deberá iniciar el servidor de seguimiento de MLflow en una terminal. Una vez que el servidor comience a ejecutarse, debería ver el siguiente resultado:

In [ ]:
# !mlflow server --host 127.0.0.1 --port 8080

[2023-11-03 15:21:37 +0100] [41485] [INFO] Starting gunicorn 21.2.0
[2023-11-03 15:21:37 +0100] [41485] [INFO] Listening at: http://127.0.0.1:8080 (41485)
[2023-11-03 15:21:37 +0100] [41485] [INFO] Using worker: sync
[2023-11-03 15:21:37 +0100] [41486] [INFO] Booting worker with pid: 41486
[2023-11-03 15:21:37 +0100] [41487] [INFO] Booting worker with pid: 41487
[2023-11-03 15:21:37 +0100] [41488] [INFO] Booting worker with pid: 41488
[2023-11-03 15:21:37 +0100] [41490] [INFO] Booting worker with pid: 41490


Recuerde mantener la terminal del sistema ejecutándose durante el tutorial, ya que al cerrarlo se cerrará el servidor.

# 3. Inicializar el MLflow Client

Para utilizar la API MLflowClient, el paso inicial consiste en importar los módulos necesarios.

In [2]:
from mlflow import MlflowClient
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor

## 3.1. Inicializando el cliente MLflow

En el paso 1 del tutorial, iniciamos un servidor de seguimiento de MLflow con:

*host* = **127.0.0.1**

*puerto* = **8080**

Nos conectamos a esa URI con MlflowClient

In [3]:
client = MlflowClient(tracking_uri="http://127.0.0.1:8080")

Imagen de la UI de MLflow:

<div style="text-align: center;">
    <img src="../images/default-ui.png" width="1200">
</div>

Como puede ver, no hay ejecuciones registradas y solo está presente el experimento predeterminado (con un ID de 0).

### 3.1.1. Buscar experimentos con la API de MLflowClient

Antes de comenzar a registrar algo en el servidor de seguimiento, echemos un vistazo a una característica clave que existe al inicio del inicio de cualquier servidor de seguimiento Mlflow: el experimento "Default".

El experimento "Default" es un marcador de posición que se utiliza para encapsular toda la información de ejecución si no se declara un experimento explícito. Mientras usa MLflow, creará nuevos experimentos para organizar proyectos, iteraciones de proyectos o agrupar lógicamente grandes actividades de modelado en una colección jerárquica agrupada. Sin embargo, si se olvida de crear un nuevo experimento antes de utilizar las capacidades de seguimiento de MLflow, el experimento "Default" es una alternativa para asegurarse de que sus datos de seguimiento no se pierdan al ejecutar una ejecución.

Lo primero que haremos es ver los metadatos asociados con los Experimentos que están en el servidor. Podemos lograr esto mediante el uso de la API `mlflow.client.MlflowClient.search_experiments()`.

Emitamos una consulta de búsqueda para ver cuáles son los resultados.

In [8]:
# Buscar experimentos sin proporcionar una query
all_experiments = client.search_experiments()

print(type(all_experiments))
print()
print(all_experiments)

<class 'mlflow.store.entities.paged_list.PagedList'>

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1699021013386, experiment_id='0', last_update_time=1699021013386, lifecycle_stage='active', name='Default', tags={}>]


Vale la pena señalar que el tipo de retorno de la API search_experiments() no es una estructura de colección básica. Más bien, es una lista de objetos Experimento. Muchos de los valores de retorno de las API de cliente de MLflow devuelven objetos que contienen atributos de metadatos asociados con la tarea que se está realizando. Este es un aspecto importante a recordar, ya que facilita la realización de secuencias de acciones más complejas, que se tratarán en tutoriales posteriores.

Con la colección devuelta, podemos iterar sobre estos objetos para acceder a los atributos de metadatos específicos del experimento predeterminado.

Para familiarizarnos con el acceso a elementos de colecciones devueltas desde las API de MLflow, extraigamos el nombre y el lifecycle_stage de la consulta search_experiments() y extraigamos estos atributos en un dict.

In [10]:
# Obtener el nombre del experimento Default, y la fase del ciclo de vida en el que se encuentra
default_experiment = [
    {"name": experiment.name, "lifecycle_stage": experiment.lifecycle_stage}
    for experiment in all_experiments
    if experiment.name == "Default"
    ][0]

pprint(default_experiment)

{'lifecycle_stage': 'active', 'name': 'Default'}


## 3.2. Crear un experimento

En esta sección, veremos:

* crear un nuevo experimento de MLflow
* aplicar metadatos en forma de etiquetas a los experimentos

Como pudimos ver antes, al acceder a la UI de MLflow, no hay ejecuciones registradas y solo está presente el experimento predeterminado (con un ID de 0).

Si bien MLflow proporciona un experimento predeterminado, sirve principalmente como una red de seguridad general para ejecuciones iniciadas sin un experimento activo específico. Sin embargo, no se recomienda su uso regular. En cambio, crear experimentos únicos para colecciones específicas de ejecuciones ofrece numerosas ventajas, como exploraremos a continuación.

Beneficios de definir experimentos únicos:

1. Organización mejorada: los experimentos le permiten agrupar ejecuciones relacionadas, lo que facilita su seguimiento y comparación. Esto es especialmente útil cuando se gestionan numerosas ejecuciones, como en proyectos a gran escala.

2. Anotación de metadatos: los experimentos pueden contener metadatos que ayuden a organizar y asociar ejecuciones con proyectos más grandes.

Considere el siguiente escenario: estamos simulando la participación en un gran proyecto de previsión de la demanda. Este proyecto implica la construcción de modelos de pronóstico para varios departamentos de una cadena de tiendas de comestibles, cada uno de los cuales alberga numerosos productos. Nuestro enfoque aquí es el departamento de “productos agrícolas”, que tiene varios elementos distintos, cada uno de los cuales requiere su propio modelo de pronóstico. Organizar estos modelos se vuelve fundamental para garantizar una fácil navegación y comparación.

In [11]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "This is the grocery forecasting project."
    "This experiment contains the produce models for apples."
)

# Provide searchable tags that define characteristics of the Runs that will be in this Experiment
experiment_tags = {
    "project_name": "grocery-forecasting",
    "store_dept": "produce",
    "team": "stores-ml",
    "project_quarter": "Q3-2023",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
produce_apples_experiment = client.create_experiment(
    name="Apple_Models", tags=experiment_tags
)

Imagen de la UI de MLflow:

<div style="text-align: center;">
    <img src="../images/experiment-page-elements.svg" width="1200">
</div>

### 3.2.1. Búsqueda basada en etiqueta

Ahora que hemos visto el experimento y entendemos cuáles de las etiquetas que especificamos durante la creación del experimento son visibles dentro de la interfaz de usuario y cuáles no, exploraremos el motivo para definir esas etiquetas a medida que aplicamos búsquedas en el servidor de seguimiento para encontrar experimentos cuyos valores de etiquetas personalizadas coincidan con nuestros términos de consulta.

Uno de los usos más versátiles de configurar etiquetas dentro de Experimentos es permitir la búsqueda de Experimentos relacionados en función de una etiqueta común. Las capacidades de filtrado dentro de la API search_experiments se pueden ver a continuación, donde buscamos experimentos cuya etiqueta personalizada project_name coincida exactamente con la previsión de comestibles.

Tenga en cuenta que el formato que se utiliza para el filtrado de búsqueda tiene algunos matices. Para entidades con nombre (por ejemplo, aquí, el término de etiquetas al principio de la cadena de filtro), las claves se pueden usar directamente. Sin embargo, para hacer referencia a etiquetas personalizadas, tenga en cuenta la sintaxis particular utilizada. Los nombres de las etiquetas personalizadas están entre comillas simples (`) y nuestra condición de búsqueda coincidente está entre comillas simples.

In [18]:
# Use search_experiments() to search on the project_name tag key
apples_experiment = client.search_experiments(
    filter_string="tags.`project_name` = 'grocery-forecasting'"
)

print(apples_experiment[0])

<Experiment: artifact_location='mlflow-artifacts:/888958100226118400', creation_time=1699023519179, experiment_id='888958100226118400', last_update_time=1699023519179, lifecycle_stage='active', name='Apple_Models', tags={'mlflow.note.content': 'This is the grocery forecasting project.This '
                        'experiment contains the produce models for apples.',
 'project_name': 'grocery-forecasting',
 'project_quarter': 'Q3-2023',
 'store_dept': 'produce',
 'team': 'stores-ml'}>


In [19]:
# Access individual tag data
print(apples_experiment[0].tags["team"])

stores-ml


## 3.3. Running our first model training

En esta sección, haremos:

* crear un conjunto de datos sintéticos que sea relevante para una tarea simple de previsión de la demanda
* iniciar una ejecución de MLflow
* registrar métricas, parámetros y etiquetas en la ejecución
* guardar el modelo en la ejecución
* registrar el modelo durante el registro del modelo

### 3.3.1. Generador de datos sintéticos para la demanda de manzanas

Tenga en cuenta que esto es únicamente para fines de demostración.

El valor de la demanda es puramente artificial y deliberadamente covariante con las características. Este no es un escenario particularmente realista del mundo real (si lo fuera, ¡no necesitaríamos científicos de datos!).

In [20]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


def generate_apple_sales_data_with_promo_adjustment(base_demand: int = 1000, n_rows: int = 5000):
    """
    Generates a synthetic dataset for predicting apple sales demand with seasonality and inflation.

    This function creates a pandas DataFrame with features relevant to apple sales.
    The features include date, average_temperature, rainfall, weekend flag, holiday flag,
    promotional flag, price_per_kg, and the previous day's demand. The target variable,
    'demand', is generated based on a combination of these features with some added noise.

    Args:
        base_demand (int, optional): Base demand for apples. Defaults to 1000.
        n_rows (int, optional): Number of rows (days) of data to generate. Defaults to 5000.

    Returns:
        pd.DataFrame: DataFrame with features and target variable for apple sales prediction.

    Example:
        >>> df = generate_apple_sales_data_with_seasonality(base_demand=1200, n_rows=6000)
        >>> df.head()
    """

    # Set seed for reproducibility
    np.random.seed(9999)

    # Create date range
    dates = [datetime.now() - timedelta(days=i) for i in range(n_rows)]
    dates.reverse()

    # Generate features
    df = pd.DataFrame(
        {
            "date": dates,
            "average_temperature": np.random.uniform(10, 35, n_rows),
            "rainfall": np.random.exponential(5, n_rows),
            "weekend": [(date.weekday() >= 5) * 1 for date in dates],
            "holiday": np.random.choice([0, 1], n_rows, p=[0.97, 0.03]),
            "price_per_kg": np.random.uniform(0.5, 3, n_rows),
            "month": [date.month for date in dates],
        }
    )

    # Introduce inflation over time (years)
    df["inflation_multiplier"] = 1 + (df["date"].dt.year - df["date"].dt.year.min()) * 0.03

    # Incorporate seasonality due to apple harvests
    df["harvest_effect"] = np.sin(2 * np.pi * (df["month"] - 3) / 12) + np.sin(
        2 * np.pi * (df["month"] - 9) / 12
    )

    # Modify the price_per_kg based on harvest effect
    df["price_per_kg"] = df["price_per_kg"] - df["harvest_effect"] * 0.5

    # Adjust promo periods to coincide with periods lagging peak harvest by 1 month
    peak_months = [4, 10]  # months following the peak availability
    df["promo"] = np.where(
        df["month"].isin(peak_months),
        1,
        np.random.choice([0, 1], n_rows, p=[0.85, 0.15]),
    )

    # Generate target variable based on features
    base_price_effect = -df["price_per_kg"] * 50
    seasonality_effect = df["harvest_effect"] * 50
    promo_effect = df["promo"] * 200

    df["demand"] = (
        base_demand
        + base_price_effect
        + seasonality_effect
        + promo_effect
        + df["weekend"] * 300
        + np.random.normal(0, 50, n_rows)
    ) * df[
        "inflation_multiplier"
    ]  # adding random noise

    # Add previous day's demand
    df["previous_days_demand"] = df["demand"].shift(1)
    df["previous_days_demand"].fillna(method="bfill", inplace=True)  # fill the first row

    # Drop temporary columns
    df.drop(columns=["inflation_multiplier", "harvest_effect", "month"], inplace=True)

    return df

In [21]:
# Generate the dataset!

data = generate_apple_sales_data_with_promo_adjustment(base_demand=1_000, n_rows=1_000)

data[-20:]

,date,average_temperature,rainfall,weekend,holiday,price_per_kg,promo,demand,previous_days_demand
980,2023-10-15 16:06:05.324877,34.130183,1.454065,1,0,1.449177,1,1501.802447,1531.085782
981,2023-10-16 16:06:05.324876,32.353643,9.462859,0,0,2.856503,1,1030.951553,1501.802447
982,2023-10-17 16:06:05.324874,18.816833,0.391470,0,0,1.326429,1,1175.352029,1030.951553
983,2023-10-18 16:06:05.324873,34.533012,2.120477,0,0,0.970131,1,1251.385504,1175.352029
984,2023-10-19 16:06:05.324872,23.057202,2.365705,0,0,1.049931,1,1203.427049,1251.385504
985,2023-10-20 16:06:05.324871,34.810165,3.089005,0,0,2.035149,1,1186.971149,1203.427049
986,2023-10-21 16:06:05.324870,29.208905,3.673292,1,0,2.518098,1,1586.249547,1186.971149
987,2023-10-22 16:06:05.324869,16.428676,4.077782,1,0,1.268979,1,1593.118915,1586.249547
988,2023-10-23 16:06:05.324868,32.067512,2.734454,0,0,0.762317,1,1252.492007,1593.118915
989,2023-10-24 16:06:05.324867,31.938203,13.883486,0,0,1.153301,1,1179.040470,1252.492007


## 3.4. Entrene y registre el modelo

Ahora que tenemos nuestro conjunto de datos y hemos visto un poco cómo se registran las ejecuciones, profundicemos en el uso de MLflow para rastrear una iteración de entrenamiento.

Para empezar, necesitaremos importar nuestros módulos necesarios.

In [22]:
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Tenga en cuenta que aquí no estamos importando MlflowClient directamente. Para esta parte, usaremos la API fluida. Las API fluidas utilizan un estado referenciado globalmente del uri del servidor de seguimiento de MLflow. Esta instancia global nos permite usar estas API de 'nivel superior' (más simples) para realizar todas las acciones que de otro modo podríamos hacer con MlflowClient, con la adición de alguna otra sintaxis útil (como los controladores de contexto que usaremos). muy pronto) para hacer que la integración de MLflow a cargas de trabajo de ML sea lo más simple posible.

Para utilizar la API fluida, necesitaremos establecer la referencia global a la dirección del servidor de seguimiento. Hacemos esto a través del siguiente comando:

In [23]:
# Use the fluent API to set the tracking uri and the active experiment
mlflow.set_tracking_uri("http://127.0.0.1:8080")

Una vez configurado esto, podemos definir algunas constantes más que usaremos al registrar nuestros eventos de entrenamiento en MLflow en forma de ejecuciones. Comenzaremos definiendo un experimento que se utilizará para registrar ejecuciones. La relación padre-hijo de Experiments to Runs y su utilidad quedará muy clara una vez que comencemos a iterar sobre algunas ideas y necesitemos comparar los resultados de nuestras pruebas.

In [26]:
# Sets the current active experiment to the "Apple_Models" experiment and returns the Experiment metadata
apple_experiment = mlflow.set_experiment("Apple_Models")

# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "apples_rf_test"

# Define an artifact path that the model will be saved to.
artifact_path = "rf_apples"

Con estas variables definidas, podemos comenzar a entrenar un modelo.

En primer lugar, veamos lo que vamos a ejecutar. Después de la visualización del código, veremos una versión anotada del código.

In [27]:
# Split the data into features and target and drop irrelevant date field and target field
X = data.drop(columns=["date", "demand"])
y = data["demand"]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    "n_estimators": 100,
    "max_depth": 6,
    "min_samples_split": 10,
    "min_samples_leaf": 4,
    "bootstrap": True,
    "oob_score": False,
    "random_state": 888,
}

# Train the RandomForestRegressor
rf = RandomForestRegressor(**params)

# Fit the model on the training data
rf.fit(X_train, y_train)

# Predict on the validation set
y_pred = rf.predict(X_val)

# Calculate error metrics
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

# Assemble the metrics we're going to write into a collection
metrics = {"mae": mae, "mse": mse, "rmse": rmse, "r2": r2}

# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:
    # Log the parameters used for the model fit
    mlflow.log_params(params)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Log an instance of the trained model for later use
    mlflow.sklearn.log_model(sk_model=rf, input_example=X_val, artifact_path=artifact_path)

/home/gmachin/miniconda3/lib/python3.10/site-packages/mlflow/models/signature.py:351: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  input_schema = _infer_schema(input_example)
/home/gmachin/miniconda3/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Para ayudar a visualizar cómo las llamadas a la API de seguimiento de MLflow se agregan a una base de código de entrenamiento de ML, consulte la siguiente figura.

<img src="../images/training-annotation.svg" width="800">

### 3.4.1. Success!

You've just logged your first MLflow model! 

Navigate to the MLflow UI to see the run that was just created (named "apples_rf_test", logged to the Experiment "Apple_Models"). 